# /Traverse & Persist with ScrapingOrchestrator

Now we have the `ScrapingOrchestrator` class that handles:
- ✅ Persist the metadata (SQLite database) and the markdown files
- ✅ Traverse the next set of child_urls
- ✅ Before scraping the next url first check if that specific page has been scraped

### Features:
1. **Database Persistence**: Uses SQLAlchemy ORM with SQLite
2. **URL Deduplication**: Tracks all scraped URLs to prevent re-scraping
3. **Queue Management**: FIFO queue for traversing child URLs
4. **Batch Processing**: Scrape single URLs or batch operations
5. **Depth Control**: Traverse URLs by depth level

In [1]:
from models import ScrapingOrchestrator
import logging
import dotenv, os, pathlib



dotenv.load_dotenv(dotenv.find_dotenv(".env"))

# Initialize the orchestrator
root_url = 'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/'
api_key = os.getenv('FIRECRAWL_API_KEY')

orchestrator = ScrapingOrchestrator(
    firecrawl_api_key=api_key,
    root_url=root_url,
    db_path=None,  # Uses default: ./data/scraping.db
    log_level=logging.DEBUG,
    ask_ollama=False  # Set to True if Ollama is running
)

print("✅ ScrapingOrchestrator initialized")
print(f"Database location: {orchestrator.db_manager.db_path}")

2026-01-17 20:41:47,434 - models.processdata.ResponseProcessor - DEBUG - Initializing ResponseProcessor
2026-01-17 20:41:47,444 - models.orchestrator.ScrapingOrchestrator - INFO - ScrapingOrchestrator initialized
2026-01-17 20:41:47,446 - models.orchestrator.ScrapingOrchestrator - ERROR - Failed loading existing scraped URLs


✅ ScrapingOrchestrator initialized
Database location: /home/joestry/git-projects/github/scraping-projects/scraping-flink-docs/firecrawl_flink_docs/data/scraping.db


### Usage Examples

#### Example 1: Scrape single URL with persistence
This scrapes a URL, saves markdown file, and persists metadata to database.

In [2]:
# Example 1: Scrape single URL and persist
# Note: You may want to use ask_ollama=False for faster testing
test_url = 'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/concepts/overview/'

metadata = orchestrator.scrape_and_persist(test_url)
if metadata:
    print(f"\n✅ Scraped and persisted!")
    print(f"   Page ID: {metadata.page_id}")
    print(f"   Title: {metadata.title}")
    print(f"   Child URLs found: {len(metadata.child_urls)}")
    print(f"   File saved: ./data/markdown_files/{metadata.prefix}_{metadata.page_id}.md")
else:
    print("URL already scraped or scrape failed")

2026-01-17 20:41:52,156 - models.processdata.ResponseProcessor - DEBUG - _normalize_url
2026-01-17 20:41:52,159 - models.processdata.ResponseProcessor - DEBUG - _normalize_url
2026-01-17 20:41:52,161 - models.orchestrator.ScrapingOrchestrator - INFO - Scraping URL
2026-01-17 20:41:53,486 - models.processdata.ResponseProcessor - INFO - parse_raw_response called
2026-01-17 20:41:53,487 - models.processdata.ResponseProcessor - DEBUG - parse_raw_response called
2026-01-17 20:41:53,489 - models.processdata.ResponseProcessor - DEBUG - content_to_hash called
2026-01-17 20:41:53,491 - models.processdata.ResponseProcessor - DEBUG - content_to_hash result
2026-01-17 20:41:53,492 - models.processdata.ResponseProcessor - DEBUG - extract_version called
2026-01-17 20:41:53,497 - models.processdata.ResponseProcessor - DEBUG - extract_version result
2026-01-17 20:41:53,498 - models.processdata.ResponseProcessor - DEBUG - extract_prefix called
2026-01-17 20:41:53,501 - models.processdata.ResponseProces

URL already scraped or scrape failed


In [3]:
print(metadata)

None


#### Example 2: Check if URL has been scraped (deduplication)

In [ ]:
# Check if URLs have been scraped
urls_to_check = [
    test_url,  # Should return True (we just scraped it)
    'https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/try-flink/datastream/',
]

for url in urls_to_check:
    has_been_scraped = orchestrator.has_been_scraped(url)
    status = "✅ Already scraped" if has_been_scraped else "❌ Not yet scraped"
    print(f"{status}: {url}")

#### Example 3: Queue child URLs for traversal

In [ ]:
# If metadata had child URLs, they are automatically added to queue
# You can also manually add URLs:

sample_child_urls = [
    ("Getting Started", "https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/try-flink/"),
    ("DataStream API", "https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/datastream/"),
]

orchestrator.add_urls_to_queue(sample_child_urls)
print(f"Queue size: {orchestrator.queue_size()}")
print(f"Queue contents:")
for i, (text, url) in enumerate(list(orchestrator.url_queue), 1):
    print(f"  {i}. {text}: {url}")

#### Example 4: Batch scraping from queue

In [ ]:
# Scrape a batch (e.g., first 3 URLs from queue)
# Note: Adjust max_urls based on your API quota
if orchestrator.queue_size() > 0:
    stats = orchestrator.scrape_batch(max_urls=3, stop_on_failure=False)
    print("\n📊 Batch Scraping Results:")
    print(f"   ✅ Scraped: {stats['scraped']}")
    print(f"   ❌ Failed: {stats['failed']}")
    print(f"   ⏭️  Skipped (already done): {stats['skipped']}")
    print(f"   📋 Queue remaining: {stats['queue_remaining']}")
else:
    print("Queue is empty! Add URLs first.")

#### Example 5: Full traversal from root (depth-limited)

In [ ]:
# Uncomment to do a full traversal (WARNING: Be careful with API quotas!)
# This will traverse all URLs starting from root up to max_depth levels
# stats = orchestrator.scrape_from_root(max_depth=2)

# For now, let's just show stats
stats = orchestrator.get_scraping_stats()
print("\n📈 Overall Scraping Statistics:")
print(f"   Root URL: {stats['root_url']}")
print(f"   Total scraped URLs: {stats['total_scraped_urls']}")
print(f"   Failed URLs: {stats['failed_urls']}")
print(f"   Pending in queue: {stats['queue_pending']}")
print(f"   Pages in database: {stats['database_pages']}")

#### Example 6: Query the database

In [ ]:
# Get all pages from database
all_pages = orchestrator.db_manager.get_all_pages()
print(f"\n📚 All pages in database ({len(all_pages)} total):")
for page in all_pages[:5]:  # Show first 5
    print(f"   - {page.title} ({page.page_id})")
    print(f"     URL: {page.url}")
    print(f"     Scraped at: {page.scrape_timestamp}")
    print()

# Get pages by version (if multiple versions exist)
pages_v120 = orchestrator.db_manager.get_pages_by_version('flink-docs-release-1.20')
print(f"Pages for Flink 1.20: {len(pages_v120)}")